In [1]:
import pandas as pd
import numpy as np
import os,  sys
import datetime as dt

import matplotlib.pyplot as plt
import matplotlib.colors as colors

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

from mpl_toolkits import mplot3d

sys.path.append('/Users/benjaminwong/JupyterNotebooks/masters_pfti/notebooks/python/')

from clean import PCBCCleaner, DrawPointAssayCleaner, DrawPointCoordCleaner

# import plotter

output_dir = "../outputs/grade_analysis_compare_dp_pcbc/"
SAVE = True

plt.rcParams.update(
    {
        'font.size': 18
    }
)

# Data Input and Cleaning

In [2]:
pcbc_df = PCBCCleaner.get_processed_data()
assay_df = DrawPointAssayCleaner.get_processed_data()
dp_coords = DrawPointCoordCleaner.get_processed_data()

# Grouping

### PCBC

In [30]:
pcbc_df

,dhid,date,weight,CU,AU
0,P08-04W,2021-02-01,1.000000,0.445853,0.473962
1,P08-04W,2021-04-01,7.185358,0.445853,0.473962
2,P08-04W,2021-06-01,90.465466,0.445853,0.473962
3,P08-04W,2021-08-01,24.556073,0.445853,0.473962
4,P08-04W,2022-01-01,2247.061308,0.529466,0.554649
...,...,...,...,...,...
13209,P26-12E,2022-09-01,638.270199,0.632717,0.543535
13210,P26-12E,2022-10-01,1362.954562,0.711671,0.608383
13211,P26-13E,2022-09-01,1.000000,0.277313,0.319020
13212,P26-13E,2022-10-01,1878.426579,0.369132,0.375707


In [12]:
dhids = list(pcbc_df['dhid'].unique())

In [13]:
pcbc_groups = {}

for dhid in dhids:
    filtered_pcbc_df = pcbc_df.query('dhid == @dhid')
    pcbc_groups[dhid] = {}
    
    try:
        pcbc_groups[dhid] = filtered_pcbc_df
    except:
        print(f'Invalid data for {dhid}')

### DP Assay

In [15]:
assay_groups = {}

for dhid in dhids:
    filtered_assay_df = assay_df.query('dhid == @dhid')
    assay_groups[dhid] = {}
    
    try:
        assay_groups[dhid] = filtered_assay_df
    except:
        print(f'Invalid data for {dhid}')

In [6]:
# assay_dhids = list(assay_df['dhid'].unique())
# assay_groups = {}

# for dhid in assay_dhids:
#     filtered_assay_df = assay_df.query('dhid == @dhid')
#     try:
#         group = {
#             'weight': filtered_assay_df['weight'],
#             'cu': filtered_assay_df['CU'],
#             'au': filtered_assay_df['AU']
#         }
#         if len(group['weight']) > 3:
#             assay_groups[dhid] = group
#     except:
#         print(f'Invalid data for {dhid}')

In [17]:
assay_groups['P20-10E']

,SAMPLEID,dhid,date,Oritype,BarcodeNo,Tons_Sampling,weight,CU,AU,AG,PB,ZN,F,C,S,SULFIDE,CNV,MPA,SamplingType
150,P20-10E_20201006,P20-10E,2020-10-06,O,U010191,NaN,27.575,2.45,2.23,10.40,0.01,0.01,NaN,NaN,NaN,NaN,NaN,NaN,Manual
345,P20-10E_20201017,P20-10E,2020-10-17,O,U010396,NaN,30.035,2.87,0.87,11.97,0.02,0.01,NaN,NaN,NaN,NaN,NaN,NaN,Manual
598,P20-10E_20201026,P20-10E,2020-10-26,O,U010687,NaN,32.485,2.70,2.51,8.90,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,Manual
794,P20-10E_20201106,P20-10E,2020-11-06,O,U010949,NaN,24.330,2.14,1.28,8.51,0.01,0.01,NaN,NaN,NaN,NaN,NaN,NaN,Manual
833,P20-10E_20201108,P20-10E,2020-11-08,O,U010996,NaN,29.025,3.13,1.97,9.49,0.01,0.01,NaN,0.10,10.89,1.70,NaN,NaN,Manual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20180,P20-10E_20220913,P20-10E,2022-09-13,O,U032673,1595.714848,32.204,0.43,0.40,2.20,0.00,0.02,NaN,NaN,NaN,NaN,NaN,NaN,Manual
20536,P20-10E_20220923,P20-10E,2022-09-23,O,U033109,1661.302720,24.206,0.04,0.04,1.43,0.02,0.06,NaN,NaN,NaN,NaN,NaN,NaN,Manual
21084,P20-10E_20221004,P20-10E,2022-10-04,O,U033683,1327.999316,26.953,0.08,0.03,1.27,0.01,0.02,NaN,NaN,NaN,NaN,NaN,NaN,Manual
21439,P20-10E_20221011,P20-10E,2022-10-11,O,U034000,672.738290,27.597,0.41,0.48,2.96,0.00,0.09,NaN,NaN,NaN,NaN,NaN,NaN,Manual


In [24]:
df = assay_df.dropna(subset=['CU','AG', 'PB', 'ZN'])
df

,SAMPLEID,dhid,date,Oritype,BarcodeNo,Tons_Sampling,weight,CU,AU,AG,PB,ZN,F,C,S,SULFIDE,CNV,MPA,SamplingType
0,P14-10W_20201002,P14-10W,2020-10-02,O,U010007,NaN,28.335,1.48,0.72,4.25,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN,Manual
1,P14-11E_20201002,P14-11E,2020-10-02,O,U010008,NaN,29.650,1.09,1.29,3.99,0.01,0.00,NaN,NaN,NaN,NaN,NaN,NaN,Manual
2,P14-11W_20201002,P14-11W,2020-10-02,O,U010009,NaN,31.135,2.73,1.61,9.87,0.01,0.03,NaN,NaN,NaN,NaN,NaN,NaN,Manual
3,P14-12W_20201002,P14-12W,2020-10-02,O,U010011,NaN,30.450,2.55,1.45,6.05,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN,Manual
4,P14-13W_20201002,P14-13W,2020-10-02,O,U010012,NaN,32.710,3.70,2.39,12.68,0.02,0.02,NaN,NaN,NaN,NaN,NaN,NaN,Manual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22376,P23-14E_20221030,P23-14E,2022-10-30,O,U035047,1374.649920,26.414,0.33,0.48,3.68,0.02,0.02,NaN,NaN,NaN,NaN,NaN,NaN,Manual
22377,P24-02E_20221030,P24-02E,2022-10-30,O,U035048,638.551200,30.239,1.98,1.29,42.00,0.00,0.08,NaN,NaN,NaN,NaN,NaN,NaN,Manual
22378,P25-10W_20221030,P25-10W,2022-10-30,O,U035049,910.884073,26.490,0.51,0.56,3.23,0.01,0.03,NaN,NaN,NaN,NaN,NaN,NaN,Manual
22379,P25-11W_20221030,P25-11W,2022-10-30,O,U035050,1443.214367,26.995,0.46,0.44,2.36,0.01,0.02,NaN,NaN,NaN,NaN,NaN,NaN,Manual


In [32]:
df = assay_df.dropna(subset=['CU','AG', 'PB', 'ZN', 'F', 'C', 'S'])

x = df[['CU','AG', 'PB', 'ZN', 'F', 'C', 'S']]
y = df['AU']
 
# with sklearn
regr = LinearRegression()
regr.fit(x, y)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

# with statsmodels
x = sm.add_constant(x) # adding a constant
 
model = sm.OLS(y, x).fit()
predictions = model.predict(x) 
 
print_model = model.summary()
print(print_model)

Intercept: 
 0.16167325109426467
Coefficients: 
 [ 0.65388221  0.01196426  1.59953691 -0.39135803]
                            OLS Regression Results                            
Dep. Variable:                     AU   R-squared:                       0.316
Model:                            OLS   Adj. R-squared:                  0.312
Method:                 Least Squares   F-statistic:                     78.01
Date:                Fri, 09 Feb 2024   Prob (F-statistic):           2.23e-54
Time:                        15:53:42   Log-Likelihood:                -865.07
No. Observations:                 679   AIC:                             1740.
Df Residuals:                     674   BIC:                             1763.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

# Outputs

### Functions

In [17]:
def calculated_linear_regression_parameters(X, y, w):
    # create linear regression model
    model = LinearRegression()

    # fit regression model
    model.fit(X, y, w)
    r_squared = model.score(X, y, w)
    m, b = model.coef_[0], model.intercept_
    
    return m, b, r_squared

def plot_au_vs_cu(ax, w, x, y, dhid=None, min_s=2, max_s=298, alpha=1):
    min_w = min(w)
    max_w = max(w)
    cmap = plt.colormaps.get('copper')
    norm = colors.Normalize(vmin=min_w, vmax=max_w)

    s = (min_s + max_s * norm(w))

    # Graph
    ax.scatter(x, y, c=cmap(norm(w)), s=s, alpha=alpha)

    ax.set_xlim(0)
    ax.set_ylim(0)

    ax.set_xlabel('CU grade (%)')
    ax.set_ylabel('AU grade (ppm)') 

In [6]:
pcbc_lr_params = {}
assay_lr_params = {}

In [28]:
for dhid, group in pcbc_groups.items():
    print(dhid)
    w, x, y = group['weight'], group['cu'], group['au']
    X = [[i] for i in x]
    
    m, b, r_squared = calculated_linear_regression_parameters(X, y, w)
    pcbc_lr_params[dhid] = {
        'r2': r_squared,
        'm': m,
        'b': b
    }

P08-04W
P08-05W
P08-06W
P08-07W
P08-08W
P09-03W
P09-04E
P09-04W
P09-05E
P09-05W
P09-06E
P09-06W
P09-07E
P09-07W
P09-08E
P09-08W
P09-09W
P09-10W
P10-03E
P10-03W
P10-04E
P10-04W
P10-05E
P10-05W
P10-06E
P10-06W
P10-07E
P10-07W
P10-08E
P10-08W
P10-09E
P10-09W
P10-10E
P10-10W
P11-03E
P11-03W
P11-04E
P11-04W
P11-05E
P11-05W
P11-06E
P11-06W
P11-07E
P11-07W
P11-08E
P11-08W
P11-09E
P11-09W
P11-10E
P11-10W
P11-11W
P11-12W
P11-13W
P12-03E
P12-03W
P12-04E
P12-04W
P12-05E
P12-05W
P12-06E
P12-06W
P12-07E
P12-07W
P12-08E
P12-08W
P12-09E
P12-09W
P12-10E
P12-10W
P12-11E
P12-11W
P12-12E
P12-12W
P12-13E
P12-13W
P13-03E
P13-03W
P13-04E
P13-04W
P13-05E
P13-05W
P13-06E
P13-06W
P13-07E
P13-07W
P13-08E
P13-08W
P13-09E
P13-09W
P13-10E
P13-10W
P13-11E
P13-11W
P13-12E
P13-12W
P13-13E
P13-13W
P13-14W
P14-03E
P14-03W
P14-04E
P14-04W
P14-05E
P14-05W
P14-06E
P14-06W
P14-07E
P14-07W
P14-08E
P14-08W
P14-09E
P14-09W
P14-10E
P14-10W
P14-11E
P14-11W
P14-12E
P14-12W
P14-13E
P14-13W
P14-14E
P14-14W
P15-03E
P15-03W
P15-04E


In [ ]:
fig = plt.figure()
fig.set_size_inches(10, 8)
ax = fig.add_subplot()

# Iterate over each drillhole
for dhid in pcbc_dhids:
    w, x, y = pcbc_groups[dhid]
    
    m, b, r_squared = calculated_linear_regression_parameters(x, y, w)
    pcbc_lr_params[dhid] = {
        'r2': r_squared,
        'm': m,
        'b': b
    }
    
    ax.cla()
    plot_au_vs_cu(ax, w, x, y, dhid=dhid)
    
    ax.set_title(f'{dhid}_AU_vs_CU_R2={r_squared:.2f}')
    ax.axline((0, b), slope=m, color='lightblue', ls='--', label=f'y = {m:.3f}x + {b:.3f}')
    ax.legend()
    
    if SAVE:
        save_loc = output_dir + f'{dhid}/'
        if not os.path.exists(save_loc):
            os.makedirs(save_loc)
        fig.savefig(save_loc + f'{dhid}_PCBC_R2={r_squared:.2f}.svg', format='svg')


# TODO: do not perform linear regression on drawpoints with fewer than 3 values
# TODO: compare PCBC and Assay linear regressions

In [33]:
## Testing
dhid = 'P08-04W'
group = pcbc_groups[dhid]
w, x, y = group['weight'], group['cu'], group['au']

X = [[i] for i in x]
m, b, r_squared = calculated_linear_regression_parameters(X, y, w)
pcbc_lr_params[dhid] = {
    'r2': r_squared,
    'm': m,
    'b': b
}
pcbc_lr_params['P08-04W']

{'r2': 0.5172634945746492, 'm': 1.3437805552238338, 'b': -0.7624937062555237}